In [12]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client import models
from qdrant_client.http.models import Distance, VectorParams, PointStruct 
from qdrant_client.models import Filter
from qdrant_client.http import models
import numpy as np
import pandas as pd
import configparser
import numpy as np

In [13]:
!export CUDA_VISIBLE_DEVICES=0,1

In [14]:
config=configparser.ConfigParser()
config.read('../config/config.cfg')

model_name = "/home/gael/.cache/torch/sentence_transformers/moussaKam_barthez/"
encoder = SentenceTransformer(model_name_or_path=model_name)

No sentence-transformers model found with name /home/gael/.cache/torch/sentence_transformers/moussaKam_barthez/. Creating a new one with MEAN pooling.


In [15]:
QDRANT_HOST=config['QDRANT']['host']
QDRANT_PORT=config['QDRANT']['port']
QDRANT_API_KEY=config['QDRANT']['qdrant_api_key']

client = QdrantClient(url=QDRANT_HOST, 
                      port=QDRANT_PORT, 
                      api_key=QDRANT_API_KEY)

In [16]:
df = pd.read_csv('../data/processed/articles.csv', index_col=0, encoding='utf-8')

In [17]:
df = df.reset_index()

In [18]:
df.columns = ['newsId', 'author', 'title', 'publishedAt', 'content']

In [19]:
def generate_item_sentence(item: pd.Series, text_columns=["title"]) -> str:
    return ' '.join([item[column] for column in text_columns])

In [20]:
df["sentence"] = df.apply(generate_item_sentence, axis=1)
df["sentence_embedding"] = df["sentence"].apply(encoder.encode)

In [21]:
client.recreate_collection(
    collection_name = "articles_fr_newsapi",
    vectors_config = models.VectorParams(
        size = encoder.get_sentence_embedding_dimension(),
        distance = models.Distance.COSINE,
    ),
)

True

In [22]:
metadata_columns = df.drop(["newsId", "sentence", "sentence_embedding"], axis=1).columns

def create_vector_point(item:pd.Series) -> PointStruct:
    """Turn vectors into PointStruct"""
    return PointStruct(
        id = item["newsId"],
        vector = item["sentence_embedding"].tolist(),
        payload = {
            field: item[field]
            for field in metadata_columns
            if (str(item[field]) not in ['None', 'nan'])
        }
    )

points = df.apply(create_vector_point, axis=1).tolist()

In [23]:
CHUNK_SIZE = 500
n_chunks = np.ceil(len(points)/CHUNK_SIZE)

for i, points_chunk in enumerate(np.array_split(points, n_chunks)):
    client.upsert(
        collection_name="articles_fr_newsapi",
        wait=True,
        points=points_chunk.tolist()
    )

In [24]:
query_text = "Macron"
query_vector = encoder.encode(query_text).tolist()

In [25]:
client.search(
    collection_name="articles_fr_newsapi",
    query_vector=query_vector,
    with_payload=["newsId", "title", "content"],
    query_filter=None
)

[ScoredPoint(id=47, version=0, score=0.46678513, payload={'content': 'La bagnole\xa0? «\xa0Moi, je ladore\xa0!\xa0», avait déclaré Emmanuel Macron lors dune interview télévisée le 24\xa0septembre. Cette passion a sans doute incité le chef de lEtat à annoncer lui-même, jeudi 14\xa0décemb… [+3356 chars]', 'title': 'Voiture électrique\xa0: une démocratisation difficile'}, vector=None, shard_key=None),
 ScoredPoint(id=130, version=0, score=0.46319127, payload={'content': "Un petit air de\xa02017, une potion de «\xa0disruption\xa0»\xa0? En nommant Gabriel Attal à Matignon, contre l'avis de sa vieille garde, Emmanuel Macron s'offre la possibilité d'une «\xa0régénération\xa0» évoquée dans… [+2473 chars]", 'title': "Macron, l'opération sang neuf"}, vector=None, shard_key=None),
 ScoredPoint(id=87, version=0, score=0.44700325, payload={'content': 'En\xa02023, le groupe japonais Toyota a conservé la tête du classement mondial des constructeurs automobiles, avec près de dix millions de voitures v

## Affichage

In [26]:
# Convert embeddings from CSV str type to NumPy Array
embedding_array = np.array(
    df['sentence_embedding'].to_list()
)

In [27]:
query = "Guerre en Ukraine"
query_embedding_response = np.array(
    encoder.encode(query)
)

In [28]:
from scipy.spatial.distance import cdist

df['distance'] = cdist(
    embedding_array,
    [query_embedding_response]
)

In [29]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(df[['distance']])

df['normalised'] = scaler.transform(df[['distance']])

In [30]:
import plotly.express as px
from sklearn.manifold import TSNE

# Create a t-SNE model
tsne_model = TSNE(
    n_components = 3,
    perplexity = 15,
    random_state = 42,
    init = 'random',
    learning_rate = 200
)
tsne_embeddings = tsne_model.fit_transform(embedding_array)

# Create a DataFrame for visualisation
visualisation_data = pd.DataFrame(
    {'x': tsne_embeddings[:, 0],
     'y': tsne_embeddings[:, 1],
     'z': tsne_embeddings[:, 2],
     'title':df['title'],
     'Similarity': df['normalised'].round(3)}
)

# Create the scatter plot using Plotly Express
plot = px.scatter_3d(
    visualisation_data,
    x = 'x',
    y = 'y',
    z = 'z',
    color = 'Similarity',
    hover_name='title',
    #color_continuous_scale = 'rainbow',
    opacity = 0.7,
    title = f"Similarity to '{query}' visualised using t-SNE"
)

plot.update_layout(
    width = 1000,
    height = 650
)

# Show the plot
plot.show()